In [ ]:
from dotenv import load_dotenv
from requests import post, get
import os
import base64
import json

load_dotenv()

#getting client_id and secret from environment variables file
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [154]:
def get_token():

    #Visit https://developer.spotify.com/documentation/web-api/tutorials/client-credentials-flow for more information

    #base64 encoding
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')

    url = 'https://accounts.spotify.com/api/token'

    headers = {'Authorization': 'Basic ' + auth_base64, 'Content-Type': 'application/x-www-form-urlencoded'}

    data = {'grant_type': 'client_credentials'}

    result = post(url, headers = headers, data = data)
    json_result = json.loads(result.content)
    token = json_result['access_token']
    return token

def get_auth_header(token):
    return {'Authorization': 'Bearer ' + token}

Function to get all songs from a playlist, returns a dictionary with track name and id

In [155]:
def get_playlist(playlist_id, limit=100, fields='items(track(id,name,external_urls))', offset=0, display=True):

    headers = get_auth_header(get_token())

    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks?fields={fields}&limit={limit}&offset={offset}'

    result = get(url, headers=headers)
    json_result = json.loads(result.content)
    track_info = json_result['items']

    track_id_list = []
    track_name_list = []
    tracks={}

    for i in range(len(track_info)):

        if(display):
            print("\n", i+1, " . ", track_info[i]['track']['name'], " | ", end=" ")
            print(track_info[i]['track']['id'], " | ", end=" ")
            print(track_info[i]['track']['external_urls']['spotify'])

        track_id_list.append(track_info[i]['track']['id'])
        track_name_list.append(track_info[i]['track']['name'])

    tracks = dict(zip(track_name_list, track_id_list))
    return tracks


# Angry Playlist - https://open.spotify.com/playlist/0KPEhXA3O9jHFtpd1Ix5OB
tracks = get_playlist(playlist_id="0KPEhXA3O9jHFtpd1Ix5OB", display=False)
print(tracks)

{'Shit Luck': '3b1UyW9VxSymsahmyL3qCD', 'Smells Like Teen Spirit': '5ghIJDpPoe3CfHMGu71E6T', 'You Know How I Do': '2OOM9QIzL3xwS0ODsjxUV4', 'Basket Case': '6L89mwZXSOwYl76YXfX13s', 'The Futile': '6Ng8VpVdgIVmJw4HY4eId9', 'Sail': '5xhQChGGhKLWqBqX4XhtYE', 'Break Stuff': '2YC6ET3q1F29B0V7UcPV70', 'I Hate Everything About You': '6rUp7v3l8yC4TKxAAR5Bmx', 'Worst Behavior': '6oF3Es1YzzmLKjGBfThUvD', 'Seventy Times 7': '78HsNTKVNADOaGxS41OSW7', "I've Had It": '5b0kcwMjAES1SKEHBxQHAJ', "I've Been Tired": '2RYe8B8dEfsqzwQYFaKNfD', 'You Oughta Know': '17ZAZ24Eyh5fKqQ06u4R3d', 'Look Back in Anger - 1999 Remaster': '5HKi6OsSg3qngxfuZeLmyX', 'Last Resort': '5eek2X5459T1HoYJk2CKXv', 'Hit Me With Your Best Shot - 1999 Digital Remaster': '7nrtHjt4Vk4Z5jMHOwaS8F', "I'm Shipping Up To Boston": '6Wze1mFZVcT3fj8hTKnBSz', 'Black Skinhead': '722tgOgdIbNe3BEyLnejw4', 'I Fought The Law': '2LWJJNFtrg6PytOdAGrxQB', 'Take Me Out': '6ooluO7DiEhI1zmK94nRCM', 'Revolution - Remastered 2009': '3UDmHZcBTQp8Iu8droNtUl'

Function to get recomendations

In [156]:
def get_recomendations(limit = 20, market = 'IN', seed_artists = None,
  seed_genres = 'classical, country', seed_tracks = None, max_acousticness = None,
  max_danceability = None, max_energy = None,
  max_instrumentalness = None, max_key = None, max_liveness = None,
  max_loudness = None, max_mode = None, max_popularity = None,
  max_speechiness = None, max_tempo = None,
  max_time_signature = None, max_valence = None,
  min_acousticness = None, min_danceability = None,
   min_energy = None,
  min_instrumentalness = None, min_key = None, min_liveness = None,
  min_loudness = None, min_mode = None, min_popularity = None,
  min_speechiness = None, min_tempo = None,
   min_valence = None,
  target_acousticness = None, target_danceability = None,
  target_energy = None, target_loudness = None,
  target_instrumentalness = None, target_key = None,
  target_liveness = None, target_mode = None,
  target_popularity = None, target_speechiness = None,
  target_tempo = None,
  target_valence = None, token = get_token()):

    headers = get_auth_header(token)

    url = 'https://api.spotify.com/v1/recommendations'

    query = f'?limit={limit}&market=IN'\
    f'{f"&target_valence={target_valence}" if target_valence else ""}'\
    f'{f"&target_loudness={target_loudness}" if target_loudness else ""}'\
    f'{f"&target_instrumentalness={target_instrumentalness}" if target_instrumentalness else ""}'\
    f'{f"&target_speechiness={target_speechiness}" if target_speechiness else ""}'\
    f'{f"&target_acousticness={target_acousticness}" if target_acousticness else ""}'\
    f'{f"&target_tempo={target_tempo}" if target_tempo else ""}'\
    f'{f"&target_liveness={target_liveness}" if target_liveness else ""}'\
    f'{f"&target_energy={target_energy}" if target_energy else ""}'\
    f'{f"&target_popularity={target_popularity}" if target_popularity else ""}'\
    f'{f"&target_danceability={target_danceability}" if target_danceability else ""}'\
    f'{f"&target_mode={target_mode}" if target_mode else ""}'\
    f'{f"&target_key={target_key}" if target_key else ""}'\
    f'{f"&seed_artists={seed_artists}" if seed_artists else ""}'\
    f'{f"&seed_genres={seed_genres}" if seed_genres else ""}'\
    f'{f"&seed_tracks={seed_tracks}" if seed_tracks else ""}'

    print(query)

    query_url = url + query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)
    print (result)
    track_info = json_result['tracks']

    for i in range(len(track_info)):
        print("\n", i+1, " . ", track_info[i]['name'], " | ", end=" ")
        print(track_info[i]['artists'][0]['name'], end=" ")
        print(track_info[i]['external_urls']['spotify'])

Function to get details about the tracks, returns dataframe

In [157]:
def get_song_details(playlist_id):

    headers = get_auth_header(get_token())

    tracks = get_playlist(playlist_id=playlist_id, display=False)

    ids = list(tracks.values())

    ids = str(ids)[1:-1].replace("'","").replace(" ","")

    url = f'https://api.spotify.com/v1/audio-features?ids={ids}'


    result = get(url, headers=headers)
    json_result = json.loads(result.content)

    df = pd.DataFrame.from_dict(json_result['audio_features'])
    return df

## Emotion - Anger

Angry Playlist data - https://open.spotify.com/playlist/0KPEhXA3O9jHFtpd1Ix5OB

In [158]:
# Angry Playlist - https://open.spotify.com/playlist/0KPEhXA3O9jHFtpd1Ix5OB
angry_df = get_song_details('0KPEhXA3O9jHFtpd1Ix5OB')
angry_df.describe()
#angry_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000
mean,0.504824,0.832676,5.323529,-5.945529,0.676471,0.110179,0.067805,0.037413,0.268194,0.500824,120.343382,207971.617647,3.941176
std,0.176102,0.113312,3.327758,2.582891,0.474858,0.090551,0.126797,0.149430,0.189280,0.176570,32.323499,58194.581451,0.238833
min,0.179000,0.435000,0.000000,-12.089000,0.000000,0.029600,0.000025,0.000000,0.045100,0.237000,77.987000,85867.000000,3.000000
25%,0.374500,0.781750,3.000000,-6.764750,0.000000,0.053925,0.001172,0.000000,0.136750,0.351000,93.649500,173646.500000,4.000000
50%,0.500000,0.858500,4.000000,-5.584500,1.000000,0.068750,0.004170,0.000008,0.242500,0.463000,113.352500,208953.000000,4.000000
75%,0.598500,0.909500,8.000000,-4.123000,1.000000,0.156750,0.082925,0.000273,0.333000,0.638750,129.938500,246847.000000,4.000000
max,0.825000,0.997000,11.000000,-1.629000,1.000000,0.340000,0.557000,0.643000,0.868000,0.954000,201.830000,331573.000000,4.000000


Angry Playlist 2 - https://open.spotify.com/playlist/4zZpffTnguBELBde2Ar30Z

In [159]:
angry_df = get_song_details('4zZpffTnguBELBde2Ar30Z')
angry_df.describe()
#angry_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.630360,0.814200,5.160000,-4.397270,0.690000,0.194950,0.064905,0.170683,0.285999,0.464223,130.549640,153851.280000,3.910000
std,0.147734,0.167916,3.876164,3.216536,0.464823,0.137746,0.090288,0.312268,0.198167,0.222155,31.951767,40656.821246,0.378594
min,0.214000,0.309000,0.000000,-11.444000,0.000000,0.035600,0.000004,0.000000,0.031100,0.039800,74.997000,82731.000000,1.000000
25%,0.564500,0.721500,1.000000,-6.631000,0.000000,0.082625,0.005347,0.000000,0.117750,0.282000,104.186750,123313.750000,4.000000
50%,0.652000,0.869500,4.500000,-4.564500,1.000000,0.152000,0.029450,0.000237,0.238000,0.492000,126.002000,150605.000000,4.000000
75%,0.734250,0.951750,9.000000,-1.451500,1.000000,0.278000,0.075300,0.108750,0.415500,0.644250,155.835500,176275.250000,4.000000
max,0.918000,0.995000,11.000000,2.044000,1.000000,0.695000,0.413000,0.966000,0.798000,0.965000,200.049000,274133.000000,4.000000


Angry Songs Recomendations Playlist 1

In [160]:
#FYI seed_genre matters a lot

get_recomendations(seed_genres="rock,metal,heavy-metal,death-metal,punk-rock", limit=15, target_popularity=100, target_danceability=0.5, target_energy=0.8, target_loudness=-5.94, target_acousticness=0.06, target_valence=0.5, target_tempo=120.34)


?limit=15&market=IN&target_valence=0.5&target_loudness=-5.94&target_acousticness=0.06&target_tempo=120.34&target_energy=0.8&target_popularity=100&target_danceability=0.5&seed_genres=rock,metal,heavy-metal,death-metal,punk-rock
<Response [200]>

 1  .  The Final Countdown  |  Europe https://open.spotify.com/track/3MrRksHupTVEQ7YbA0FsZK

 2  .  In the End  |  Linkin Park https://open.spotify.com/track/60a0Rd6pjrkxjPbaKzXjfq

 3  .  The Philosopher  |  Death https://open.spotify.com/track/7zKweXON2KcgXizqUcu7lf

 4  .  Your Treachery Will Die WIth You  |  Dying Fetus https://open.spotify.com/track/4FZoFK86X6Wd1csU9afTIA

 5  .  I Hate Everything About You  |  Three Days Grace https://open.spotify.com/track/6rUp7v3l8yC4TKxAAR5Bmx

 6  .  Domination  |  Pantera https://open.spotify.com/track/769cLRTw2y6KRdkFWFkxtu

 7  .  Chopped in Half  |  Obituary https://open.spotify.com/track/01cGujYWGF7JchJLSgf6Ta

 8  .  You've Got Another Thing Coming  |  Judas Priest https://open.spotify.com/track/

Angry Song Recommendation Playlist 2

In [161]:
get_recomendations(seed_genres="rock,metal,heavy-metal,death-metal,punk-rock", limit=15, target_popularity=90, target_danceability=0.73, target_energy=0.9, target_loudness=-5.39, target_acousticness=0.06, target_valence=0.47, target_tempo=150.34)

?limit=15&market=IN&target_valence=0.47&target_loudness=-5.39&target_acousticness=0.06&target_tempo=150.34&target_energy=0.9&target_popularity=90&target_danceability=0.73&seed_genres=rock,metal,heavy-metal,death-metal,punk-rock
<Response [200]>

 1  .  Overactive Imagination  |  Death https://open.spotify.com/track/7gRKe3yTuARYOJwF6ecGzg

 2  .  Subjected to a Beating  |  Dying Fetus https://open.spotify.com/track/7dU9Z24yR9AKuS9cAe2Tj7

 3  .  Sex on Fire  |  Kings of Leon https://open.spotify.com/track/0ntQJM78wzOLVeCUAW7Y45

 4  .  Sugar, We're Goin Down  |  Fall Out Boy https://open.spotify.com/track/2TfSHkHiFO4gRztVIkggkE

 5  .  Sacrificial Suicide  |  Deicide https://open.spotify.com/track/2kW45RgvG6cGkyAfq9ZlsZ

 6  .  Symphony Of Destruction  |  Megadeth https://open.spotify.com/track/4YJZjINIhakR98fnz1Pvup

 7  .  Sacred Serenity  |  Death https://open.spotify.com/track/5I2nvva658EdhIAtRQadWG

 8  .  Girls & Boys  |  Good Charlotte https://open.spotify.com/track/65mxB9IWpmVo4

## Emotion - Sad

Sad Playlist 1 - https://open.spotify.com/playlist/6yYA6aUGp8qUTgQWWYkPkP

In [162]:
sad_df = get_song_details('6yYA6aUGp8qUTgQWWYkPkP')
sad_df.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
mean,0.512253,0.371061,5.666667,-9.330869,0.696970,0.061566,0.676262,0.009802,0.144085,0.274463,114.720727,215402.646465,3.888889
std,0.126681,0.163549,3.395676,3.416102,0.461907,0.067402,0.262760,0.067509,0.095173,0.128959,32.656494,33608.011737,0.401245
min,0.236000,0.056100,0.000000,-23.023000,0.000000,0.027400,0.046700,0.000000,0.065300,0.078400,67.016000,144377.000000,3.000000
25%,0.396000,0.273500,4.000000,-10.773500,0.000000,0.032000,0.567000,0.000000,0.097400,0.169500,83.702500,196225.000000,4.000000
50%,0.507000,0.363000,6.000000,-8.893000,1.000000,0.038200,0.766000,0.000000,0.113000,0.250000,111.893000,212693.000000,4.000000
75%,0.615000,0.459500,9.000000,-7.251000,1.000000,0.053500,0.887000,0.000026,0.144500,0.364000,138.945500,234886.500000,4.000000
max,0.791000,0.888000,11.000000,-3.536000,1.000000,0.438000,0.978000,0.657000,0.764000,0.621000,180.917000,298899.000000,5.000000


Jazzy Sad Songs almost romanticised

In [163]:
get_recomendations(seed_genres="emo,blues", limit=15, target_popularity=90, target_danceability=0.48, target_energy=0.32, target_loudness=-10.49, target_acousticness=0.76, target_valence=0.11, target_tempo=115)

?limit=15&market=IN&target_valence=0.11&target_loudness=-10.49&target_acousticness=0.76&target_tempo=115&target_energy=0.32&target_popularity=90&target_danceability=0.48&seed_genres=emo,blues
<Response [200]>

 1  .  St James Infirmary  |  Hugh Laurie https://open.spotify.com/track/4eAlHTYUZvllH2dE2juVQZ

 2  .  Gravity  |  John Mayer https://open.spotify.com/track/3SktMqZmo3M9zbB7oKMIF7

 3  .  I Won't Give Up  |  Jason Mraz https://open.spotify.com/track/05pKAafT85jeeNhZ6kq7HT

 4  .  Give Me One Reason  |  Tracy Chapman https://open.spotify.com/track/4vDBJeeQCbhP9FaPPMsYkY

 5  .  Brown Sugar  |  ZZ Top https://open.spotify.com/track/5H0buOvucTuvEvrJFxtftl

 6  .  All of Me  |  John Legend https://open.spotify.com/track/3U4isOIWM3VvDubwSI3y7a

 7  .  Misguided Ghosts  |  Paramore https://open.spotify.com/track/2VYqkb77OjToeRAnMNd4yU

 8  .  Drown in My Own Tears  |  Ray Charles https://open.spotify.com/track/0oQtjAjUeksVd9gHE5fl02

 9  .  The Scientist  |  Coldplay https://open.spot

Sad Playlist 2 - https://open.spotify.com/playlist/6vEYfoIeIIzC87k94JDcYj

In [164]:
sad_df = get_song_details('6vEYfoIeIIzC87k94JDcYj')
sad_df.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.512000,0.469166,5.820000,-9.39252,0.420000,0.043229,0.581261,0.053797,0.125796,0.356518,110.435710,238915.920000,3.750000
std,0.131753,0.200132,3.497127,4.46723,0.496045,0.028186,0.277161,0.202424,0.062019,0.177594,28.561243,74372.176864,0.729951
min,0.198000,0.017400,0.000000,-29.85000,0.000000,0.023700,0.018900,0.000000,0.057700,0.041900,64.934000,41750.000000,1.000000
25%,0.408750,0.365000,3.000000,-10.93450,0.000000,0.030700,0.367750,0.000000,0.091825,0.234000,86.449000,198120.500000,4.000000
50%,0.535000,0.486500,5.000000,-8.54450,0.000000,0.035400,0.631000,0.000004,0.111000,0.338000,102.183500,242777.000000,4.000000
75%,0.616250,0.626500,9.000000,-6.61075,1.000000,0.044600,0.820000,0.000193,0.133500,0.474000,131.104250,276900.500000,4.000000
max,0.761000,0.843000,11.000000,-3.29300,1.000000,0.272000,0.988000,0.972000,0.420000,0.802000,180.042000,470500.000000,5.000000


Quite Sad Lyrics (giving sad genre gives genuinely depressing lyrics however they're often anger sad, may or may not use acooustic)

In [182]:
get_recomendations(seed_genres="pop,emo,blues,acoustic,grunge", limit=15, target_popularity=90, target_danceability=0.51, target_energy=0.46, target_loudness=-9.39, target_acousticness=0.58, target_valence=0.35, target_tempo=110.4)

?limit=15&market=IN&target_valence=0.35&target_loudness=-9.39&target_acousticness=0.58&target_tempo=110.4&target_energy=0.46&target_popularity=90&target_danceability=0.51&seed_genres=pop,emo,blues,acoustic,grunge
<Response [200]>

 1  .  Juke Jam (feat. Justin Bieber & Towkio)  |  Chance the Rapper https://open.spotify.com/track/3eze1OsZ1rqeXkKStNfTmi

 2  .  All Apologies  |  Nirvana https://open.spotify.com/track/3bH8PZsfbCRIoeNiFm7JmU

 3  .  Electric (feat. Khalid)  |  Alina Baraz https://open.spotify.com/track/2mj7x99sNUsVHPBRK51zlG

 4  .  The Scientist  |  Coldplay https://open.spotify.com/track/75JFxkI2RXiU7L9VXzMkle

 5  .  Good Riddance (Time of Your Life)  |  Green Day https://open.spotify.com/track/6ORqU0bHbVCRjXm9AjyHyZ

 6  .  Sweet Creature  |  Harry Styles https://open.spotify.com/track/5kRPPEWFJIMox5qIkQkiz5

 7  .  Red House  |  Jimi Hendrix https://open.spotify.com/track/3PwKz7ML8VnNAp15OzIMWG

 8  .  I Just Want To Make Love To You  |  Etta James https://open.spotif

## Emotion - Happy

Happy Playlist 1 - https://open.spotify.com/playlist/0jrlHA5UmxRxJjoykf7qRY

In [166]:
happy_df = get_song_details('0jrlHA5UmxRxJjoykf7qRY')
happy_df.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,55.00000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000
mean,0.62160,0.621655,4.963636,-6.248455,0.745455,0.060415,0.260230,0.000676,0.155238,0.492613,116.663382,216665.927273,3.890909
std,0.15312,0.176409,3.839192,2.553548,0.439620,0.049270,0.248711,0.003703,0.140265,0.242538,26.940976,32622.796627,0.458368
min,0.23000,0.128000,0.000000,-18.717000,0.000000,0.023200,0.001730,0.000000,0.034400,0.038200,68.534000,156205.000000,1.000000
25%,0.53750,0.521500,1.000000,-6.937000,0.500000,0.034350,0.064300,0.000000,0.095950,0.340000,96.463500,195720.000000,4.000000
50%,0.67200,0.640000,5.000000,-6.049000,1.000000,0.045200,0.180000,0.000000,0.115000,0.486000,114.988000,211560.000000,4.000000
75%,0.72900,0.762500,8.000000,-4.789500,1.000000,0.065950,0.421000,0.000008,0.165500,0.657500,132.497500,241801.000000,4.000000
max,0.85600,0.884000,11.000000,-2.278000,1.000000,0.276000,0.894000,0.026000,0.988000,0.952000,186.003000,290840.000000,4.000000


white club dance happy songs, however if you remove the current genres and use indian, it suggested chikni chameli which is always a good sign

In [167]:
get_recomendations(seed_genres="happy,dance,funk", limit=15, target_popularity=90, target_danceability=0.75, target_energy=0.76, target_loudness=-4.24, target_acousticness=0.32, target_valence=0.49, target_tempo=116.4)

?limit=15&market=IN&target_valence=0.49&target_loudness=-4.24&target_acousticness=0.32&target_tempo=116.4&target_energy=0.76&target_popularity=90&target_danceability=0.75&seed_genres=happy,dance,funk
<Response [200]>

 1  .  Everything  |  Michael Bublé https://open.spotify.com/track/4T6HLdP6OcAtqC6tGnQelG

 2  .  Show Me Love  |  Robin S https://open.spotify.com/track/4t0UsYzmmmZRMTWn77jiGF

 3  .  The Fox (What Does the Fox Say?)  |  Ylvis https://open.spotify.com/track/5HOpkTTVcmZHnthgyxrIL8

 4  .  California Gurls  |  Katy Perry https://open.spotify.com/track/6tS3XVuOyu10897O3ae7bi

 5  .  Electric Feel  |  MGMT https://open.spotify.com/track/3FtYbEfBqAlGO46NUDQSAt

 6  .  Outside (feat. Ellie Goulding)  |  Calvin Harris https://open.spotify.com/track/7MmG8p0F9N3C4AXdK6o6Eb

 7  .  Too Hot To Stop (Part 1)  |  The Bar-Kays https://open.spotify.com/track/3oJL3rNbTyRmHaU2Yr4TpP

 8  .  Marry You  |  Bruno Mars https://open.spotify.com/track/6SKwQghsR8AISlxhcwyA9R

 9  .  Pocketful o

In [168]:
get_recomendations(seed_genres="dancehall,hip-hop,pop,guitar", limit=15, target_popularity=100, target_danceability=0.66, target_energy=0.765, target_loudness=-4.81, target_acousticness=0.01, target_valence=0.56, target_tempo=186.4, target_liveness=0.35)

?limit=15&market=IN&target_valence=0.56&target_loudness=-4.81&target_acousticness=0.01&target_tempo=186.4&target_liveness=0.35&target_energy=0.765&target_popularity=100&target_danceability=0.66&seed_genres=dancehall,hip-hop,pop,guitar
<Response [200]>

 1  .  Black and Yellow  |  Wiz Khalifa https://open.spotify.com/track/5A6OHHy73AR5tLxgTc98zz

 2  .  Roar  |  Katy Perry https://open.spotify.com/track/6F5c58TMEs1byxUstkzVeM

 3  .  Either Way (feat. Joey Bada$$)  |  Snakehips https://open.spotify.com/track/3nodtPfXcok53E8Uze8gvD

 4  .  Powerglide (feat. Juicy J) - From SR3MM  |  Rae Sremmurd https://open.spotify.com/track/1BuZAIO8WZpavWVbbq3Lci

 5  .  No Games  |  Serani https://open.spotify.com/track/3WU2blK7YBcugEEU9CATvF

 6  .  679 (feat. Remy Boyz)  |  Fetty Wap https://open.spotify.com/track/5NQbUaeTEOGdD6hHcre0dZ

 7  .  How You Remind Me  |  Nickelback https://open.spotify.com/track/0gmbgwZ8iqyMPmXefof8Yf

 8  .  Collard Greens  |  ScHoolboy Q https://open.spotify.com/track/

## Emotion - Fear

Twisted horror Playlist - https://open.spotify.com/playlist/5AM4lgcUAw5sokybXj3ny7

In [169]:
fear_df = get_song_details('5AM4lgcUAw5sokybXj3ny7')
fear_df.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000
mean,0.610103,0.740590,5.346154,-5.101808,0.423077,0.093497,0.113711,0.007837,0.197541,0.492500,121.161564,205913.294872,3.974359
std,0.125681,0.132894,3.635421,1.670068,0.497245,0.073925,0.172967,0.045588,0.140285,0.232236,30.256336,37421.896899,0.159085
min,0.342000,0.396000,0.000000,-9.348000,0.000000,0.028600,0.000247,0.000000,0.043300,0.061400,68.039000,109302.000000,3.000000
25%,0.550250,0.642750,2.000000,-6.223000,0.000000,0.043150,0.012000,0.000000,0.105000,0.321000,95.041750,185142.250000,4.000000
50%,0.593500,0.771000,5.000000,-5.010500,0.000000,0.067250,0.046650,0.000000,0.149000,0.506500,120.008500,202357.000000,4.000000
75%,0.700250,0.846250,8.000000,-3.958500,1.000000,0.105750,0.121500,0.000051,0.266000,0.630250,143.746500,222025.750000,4.000000
max,0.927000,0.965000,11.000000,-0.818000,1.000000,0.325000,0.870000,0.388000,0.799000,0.967000,190.109000,363521.000000,4.000000


Hard to decide genres

In [178]:
get_recomendations(seed_genres="alternative,indie,punk", limit=15, target_popularity=90, target_danceability=0.61, target_energy=0.74, target_loudness=-5.101, target_acousticness=0.113, target_valence=0.49, target_tempo=121.1)

?limit=15&market=IN&target_valence=0.49&target_loudness=-5.101&target_acousticness=0.113&target_tempo=121.1&target_energy=0.74&target_popularity=90&target_danceability=0.61&seed_genres=alternative,indie,punk
<Response [200]>

 1  .  Brazil  |  Declan McKenna https://open.spotify.com/track/4sNG6zQBmtq7M8aeeKJRMQ

 2  .  Little Talks  |  Of Monsters and Men https://open.spotify.com/track/2ihCaVdNZmnHZWt0fvAM7B

 3  .  On Melancholy Hill  |  Gorillaz https://open.spotify.com/track/0q6LuUqGLUiCPP1cbdwFs3

 4  .  Heaven Knows I'm Miserable Now - 2011 Remaster  |  The Smiths https://open.spotify.com/track/1xaTREM89RbIxkcjlpf4Uw

 5  .  Lying Is the Most Fun a Girl Can Have Without Taking Her Clothes Off  |  Panic! At The Disco https://open.spotify.com/track/7wOmQJeVX6qjNXqqsKOkPx

 6  .  Like a Stone  |  Audioslave https://open.spotify.com/track/3YuaBvuZqcwN3CEAyyoaei

 7  .  Daylight  |  Matt and Kim https://open.spotify.com/track/57i3M29DWoo7RDk0Tf0LZG

 8  .  Because the Night  |  Patti S

Horror Playlist 2 - https://open.spotify.com/playlist/70c2IG67uDLGIiJ3sJH76k

In [179]:
fear_df = get_song_details('70c2IG67uDLGIiJ3sJH76k')
fear_df.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,57.000000,57.000000,57.000000,57.00000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000
mean,0.532667,0.398219,5.859649,-13.50186,0.578947,0.082156,0.586421,0.457874,0.179126,0.362244,108.768491,170717.771930,3.789474
std,0.192983,0.238618,3.382933,6.08021,0.498117,0.094926,0.365103,0.408676,0.126617,0.301974,25.686681,86087.613648,0.452645
min,0.120000,0.020600,0.000000,-25.69400,0.000000,0.025700,0.000779,0.000000,0.071100,0.032700,61.007000,32014.000000,3.000000
25%,0.396000,0.186000,3.000000,-18.01100,0.000000,0.037500,0.259000,0.003120,0.098100,0.076700,91.470000,104973.000000,4.000000
50%,0.569000,0.411000,7.000000,-14.06800,1.000000,0.049100,0.687000,0.388000,0.120000,0.260000,110.023000,148040.000000,4.000000
75%,0.690000,0.539000,8.000000,-9.08100,1.000000,0.079600,0.911000,0.874000,0.222000,0.630000,127.965000,209829.000000,4.000000
max,0.881000,0.919000,11.000000,-1.87000,1.000000,0.507000,0.996000,0.971000,0.648000,0.967000,175.236000,402952.000000,5.000000


Easier to give users calming songs when they feel fearful - https://open.spotify.com/playlist/0cj48sijCRDJ3Hatx1k1vJ

In [187]:
fear_df = get_song_details('0cj48sijCRDJ3Hatx1k1vJ')
fear_df.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.00000
mean,0.593500,0.547063,5.953125,-7.856656,0.718750,0.063233,0.432146,0.030521,0.180677,0.406300,123.121719,192055.453125,3.71875
std,0.144844,0.206091,3.760402,3.761413,0.453163,0.049715,0.303651,0.123513,0.141958,0.216101,30.195238,51912.501504,0.57649
min,0.332000,0.020300,0.000000,-23.163000,0.000000,0.025600,0.000038,0.000000,0.044700,0.089900,76.074000,63664.000000,1.00000
25%,0.467500,0.422000,3.000000,-9.246750,0.000000,0.038250,0.201500,0.000000,0.099500,0.227250,101.571750,167804.750000,3.00000
50%,0.598000,0.576000,6.500000,-7.214500,1.000000,0.048950,0.376500,0.000003,0.116000,0.376500,116.432000,184900.000000,4.00000
75%,0.723000,0.667750,9.250000,-5.338750,1.000000,0.065050,0.655000,0.000693,0.206750,0.551250,144.017750,220258.250000,4.00000
max,0.872000,0.941000,11.000000,-2.958000,1.000000,0.334000,0.962000,0.743000,0.696000,0.867000,202.466000,357931.000000,5.00000


In [200]:
get_recomendations(seed_genres="study,chill,piano", limit=15, target_popularity=90, target_danceability=0.46, target_energy=0.44, target_loudness=-7.9, target_acousticness=0.43, target_valence=0.60, target_tempo=123.1, target_instrumentalness=0.03)

?limit=15&market=IN&target_valence=0.6&target_loudness=-7.9&target_instrumentalness=0.03&target_acousticness=0.43&target_tempo=123.1&target_energy=0.44&target_popularity=90&target_danceability=0.46&seed_genres=study,chill,piano
<Response [200]>

 1  .  Vienna  |  Billy Joel https://open.spotify.com/track/4U45aEWtQhrm8A5mxPaFZ7

 2  .  Another Love  |  Tom Odell https://open.spotify.com/track/3JvKfv6T31zO0ini8iNItO

 3  .  Lucky  |  Jason Mraz https://open.spotify.com/track/0IktbUcnAGrvD03AWnz3Q8

 4  .  If I Ain't Got You  |  Alicia Keys https://open.spotify.com/track/3XVBdLihbNbxUwZosxcGuJ

 5  .  Good Riddance (Time of Your Life)  |  Green Day https://open.spotify.com/track/6ORqU0bHbVCRjXm9AjyHyZ

 6  .  Daughters  |  John Mayer https://open.spotify.com/track/5FPnjikbwlDMULCCCa6ZCJ

 7  .  Let It Go - From "Frozen"/Soundtrack Version  |  Idina Menzel https://open.spotify.com/track/0qcr5FMsEO85NAQjrlDRKo

 8  .  Viva La Vida  |  Coldplay https://open.spotify.com/track/1mea3bSkSGXuIRvn